In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# train.csv : 훈련용
# test.csv : 테스트용 -> 예측해서 제출
# sample_sub...csv : 제출용
import os

path = '/Users/sehyunjeon/Desktop/playground-series-s4e1/raw data'
train = pd.read_csv( os.path.join(path,'train.csv'), index_col='id' )
test  = pd.read_csv( os.path.join(path,'test.csv'), index_col='id' )
submission = pd.read_csv( os.path.join(path,'sample_submission.csv') )

display( train.head(1)) # 0 ~ 165033
display( test.head(1)) # 165034 ~
display( submission.head(1) ) # 0 혹은 1일 확률(predict_proba())을 제시-> 기입 -> 제출

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
id,,,,,,,,,,,,,
0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.0,2,1.0,0.0,181449.97,0


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
id,,,,,,,,,,,,
165034,15773898,Lucchese,586,France,Female,23.0,2,0.0,2,0.0,1.0,160976.75


,id,Exited
0,165034,0.5


In [3]:
# 결측치 수 계산
# True, False => sum() => 1, 0 치환되서 처리

train.isnull().sum().values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [4]:
# 고유값 수 계산
train.nunique()

CustomerId         23221
Surname             2797
CreditScore          457
Geography              3
Gender                 2
Age                   71
Tenure                11
Balance            30075
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary    55298
Exited                 2
dtype: int64

In [5]:
def summary_feature_info( df ):
    '''
        주어진 df로 부터 피처별 요약된 정보 생성 제공
    '''
    # 타입을 데이터로 기본 구성
    summary_df = pd.DataFrame( df.dtypes, columns=['타입'])
    summary_df.reset_index(inplace=True) # 인덱스 -> 컬럼으로 이동
    summary_df.rename( columns={'index':'feature'}, inplace=True) # 컬럼명 변경
    #결측치수
    summary_df['결측치수'] = df.isnull().sum().values
    # 고유값수
    summary_df['고유값수'] = df.nunique().values
    # 샘플함수를 이용 추출후 하나씩 적용해도 관계 없음
    # 샘플값_0
    summary_df['샘플값_0'] = df.iloc[0].values
    # 샘플값_1
    summary_df['샘플값_1'] = df.iloc[1].values
    # 샘플값_2
    summary_df['샘플값_2'] = df.iloc[2].values
    return summary_df


# 피처요약표 생성
summary_feature_info( train )

,feature,타입,결측치수,고유값수,샘플값_0,샘플값_1,샘플값_2
0,CustomerId,int64,0,23221,15674932,15749177,15694510
1,Surname,object,0,2797,Okwudilichukwu,Okwudiliolisa,Hsueh
2,CreditScore,int64,0,457,668,627,678
3,Geography,object,0,3,France,France,France
4,Gender,object,0,2,Male,Male,Male
5,Age,float64,0,71,33.0,33.0,40.0
6,Tenure,int64,0,11,3,1,10
7,Balance,float64,0,30075,0.0,0.0,0.0
8,NumOfProducts,int64,0,4,2,2,2
9,HasCrCard,float64,0,2,1.0,1.0,1.0


In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from category_encoders import CatBoostEncoder

# Min-Max Scale (train-test 데이터 포함)
scale_columns = ['CreditScore', 'Age', 'Balance', 'EstimatedSalary']

def scale_data(train, test, column):
    scaler = MinMaxScaler()
    max_value = max(train[column].values.max(), test[column].values.max())
    min_value = min(train[column].values.min(), test[column].values.min())

    train[column] = (train[column] - min_value)/(max_value - min_value)
    test[column] = (test[column] - min_value)/(max_value - min_value)

    return train, test

for column in scale_columns:
    train, test = scale_data(train, test, column)

# Feature 특징을 포함한 열 추가
def FeatureExtraction(df): 
    df['IsSenior'] = df['Age'].apply(lambda x: 1 if x >= 60 else 0) # 고령고객
    df['IsActive_by_CreditCard'] = df['HasCrCard'] * df['IsActiveMember'] # 신용카드-활동고객 여부
    df['Products_Per_Tenure'] =  df['Tenure'] / df['NumOfProducts'] # 이용하는 상품 대비 이용기간(년)
    df['AgeCategory'] = np.round(df.Age/20).astype('int').astype('category') # 나이 카테고리 생성
    df['Sur_Geo_Gend_Sal'] = df['Surname']+df['Geography']+df['Gender']+np.round(df.EstimatedSalary).astype('str') # 카테고리형 변수들끼리 결합한 열 생성
    return df

train = FeatureExtraction(train)
test = FeatureExtraction(test)

In [7]:
# 저장
train.to_csv("/Users/sehyunjeon/Desktop/playground-series-s4e1/data_processed2/train_processed.csv",index=False)
test.to_csv("/Users/sehyunjeon/Desktop/playground-series-s4e1/data_processed2/test_processed.csv",index=False)

In [8]:
Category_Columns = ["Geography", "Gender", "IsSenior", "IsActive_by_CreditCard", "Products_Per_Tenure", "AgeCategory"]

# 카테고리형 변수 인코딩
for column in Category_Columns:
    le = LabelEncoder()
    train[column] = le.fit_transform(train[column])
    test[column] = le.transform(test[column])

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# TF-IDF를 사용하여 Surname 열을 벡터화한 후 SVD를 사용하여 차원 축소
vectorizer = TfidfVectorizer(max_features=1000)
vectors_train=vectorizer.fit_transform(train['Surname'])
vectors_test=vectorizer.transform(test['Surname'])

svd = TruncatedSVD(10)
x_pca_train = svd.fit_transform(vectors_train)
x_pca_test = svd.transform(vectors_test)
tfidf_df_train = pd.DataFrame(x_pca_train)
tfidf_df_test = pd.DataFrame(x_pca_test)

# TF-IDF로 벡터화된 성씨 데이터의 각 특징에 대한 열 이름 생성
# 그리고 이를 기존 데이터프레임에 추가하여 학습 데이터에 새로운 열 추가
cols = [('surname'+"_tfidf_"+str(f)) for f in tfidf_df_train.columns]
tfidf_df_train.columns = cols
tfidf_df_test.columns = cols
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
train = pd.concat([train, tfidf_df_train], axis=1)
test = pd.concat([test, tfidf_df_test], axis=1)

# CatBoostEncoder()를 사용하여 Surame 열 인코딩
encoder = CatBoostEncoder()
train['Surname'] = encoder.fit_transform(train['Surname'],train['Exited'])
test['Surname'] = encoder.transform(test['Surname'])

# CatBoostEncoder()를 사용하여 Sur_Geo_Gend_Sal 열 인코딩
encoder = CatBoostEncoder()
train['Sur_Geo_Gend_Sal'] = encoder.fit_transform(train['Sur_Geo_Gend_Sal'],train['Exited'])
test['Sur_Geo_Gend_Sal'] = encoder.transform(test['Sur_Geo_Gend_Sal'])

train.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,...,surname_tfidf_0,surname_tfidf_1,surname_tfidf_2,surname_tfidf_3,surname_tfidf_4,surname_tfidf_5,surname_tfidf_6,surname_tfidf_7,surname_tfidf_8,surname_tfidf_9
0,15674932,0.211599,0.636,0,1,0.202703,3,0.000000,2,1.0,...,-1.544090e-07,-2.746845e-07,-4.762159e-07,5.157937e-07,-7.900809e-07,-3.239333e-06,0.000005,0.000015,-0.000025,-0.000008
1,15749177,0.211599,0.554,0,1,0.202703,1,0.000000,2,1.0,...,-1.093689e-08,4.533202e-07,-1.388939e-06,-1.680378e-06,4.667408e-07,-2.415699e-07,0.000008,-0.000012,-0.000022,0.000007
2,15694510,0.211599,0.656,0,1,0.297297,10,0.000000,2,1.0,...,1.731739e-04,-3.857258e-03,-1.213762e-03,-6.038890e-03,-2.209604e-03,-6.966318e-03,-0.008642,0.005035,0.171440,0.618380
3,15741417,0.211599,0.462,0,1,0.216216,2,0.593398,1,1.0,...,4.369681e-05,-1.070914e-03,7.032397e-04,1.710856e-04,-1.483213e-03,-2.139818e-03,0.791203,0.606838,-0.025134,-0.000423
4,15766172,0.211599,0.732,2,1,0.202703,5,0.000000,2,1.0,...,-5.463089e-08,5.351728e-07,4.722395e-07,-2.026032e-07,1.646832e-06,-4.734056e-07,0.000002,-0.000004,-0.000005,0.000008


In [11]:
# CustomerId 열은 단지 고유 식별자이므로 삭제
train.drop(columns=["CustomerId"],inplace=True)
test.drop(columns=["CustomerId"],inplace=True)

In [12]:
# 인덱스 확인
print(train.index)
print(test.index)

RangeIndex(start=0, stop=165034, step=1)
RangeIndex(start=0, stop=110023, step=1)


In [13]:
# 저장
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

train.to_csv("/Users/sehyunjeon/Desktop/playground-series-s4e1/data_processed2/bank_train.csv", index=False)
test.to_csv("/Users/sehyunjeon/Desktop/playground-series-s4e1/data_processed2/bank_test.csv", index=False)